
- About the parameters
Two parameters for the location on the sky: Alpha, Delta
Three parameters for the orientation of the source: cosi, psi, phi
One parameter that describes the "strength" of the signal: h0
Two intrinsic parameters: frequency and spin-down: F0, F1

-  FFTs
Hi.Is there a fixed window length for fourier transform？

For this kind of signal the recommended lenght is 1800seconds (30 mins). There might be gaps in the timestamp but the considered length is 1800 seconds. Rodrigo can confirm better. :)

Yes, each timestamp labels a period of time of 1800s (you can check that by noting the frequency resolution of 1/1800 Hz), but timestamps need not to be consecutive (i.e. there may be times at which no data is collected).

- About targets 

Hey Rodrigo Tenorio, thank you for providing resources for generating more data. I am unclear about the "target" part while generating the data. The kenrel provided to generate the data does not seem to the information about the label (target 1, 0, or -1). Or am I missing something? Thanks in advance.

0/1 labels refer to whether we included a simulated signal to that sample (1) or if it consists only on noise.
In terms of the quantities of the kernel your refer to, a label of 1 would correspond to a signal with an amplitude h0 greater than 0 (i. e. there's a signal), while a label of 0 would correspond to not having a signal (i.e. amplitude h0 = 0).
The same definition can be made in terms of SNR: Label of 1 corresponds to SNR > 0, label 0 corresponds to SNR = 0 (no signal added at all).

In [1]:
import os
import sys

import numpy as np
import matplotlib.pyplot as plt

import pyfstat

from scipy import stats
from tqdm import tqdm

22-11-05 09:40:46.721 pyfstat INFO    : Running PyFstat version 1.18.1+1.73ad1acd.clean
22-11-05 09:40:47.091 pyfstat.utils.importing INFO    : No $DISPLAY environment variable found, so importing matplotlib.pyplot with non-interactive 'Agg' backend.


In [2]:

def combine_amplitudes(amplitude_0, amplitude_1):
    # Get the real part of the amplitudes
    real_0 = amplitude_0.real
    real_1 = amplitude_1.real
    # Get the imaginary part of the amplitudes
    imag_0 = amplitude_0.imag
    imag_1 = amplitude_1.imag
    # normalize the amplitudes to be between 0 and 1
    real_0 = (real_0 - real_0.min()) / (real_0.max() - real_0.min())
    real_1 = (real_1 - real_1.min()) / (real_1.max() - real_1.min())
    imag_0 = (imag_0 - imag_0.min()) / (imag_0.max() - imag_0.min())
    imag_1 = (imag_1 - imag_1.min()) / (imag_1.max() - imag_1.min())
    # Expand the dims of the real and imaginary parts
    real_0 = np.expand_dims(real_0, axis=0)
    real_1 = np.expand_dims(real_1, axis=0)
    imag_0 = np.expand_dims(imag_0, axis=0)
    imag_1 = np.expand_dims(imag_1, axis=0)
    # Make a vstack of the amplitudes
    amplitudes = np.vstack((real_0, imag_0, real_1, imag_1))
    return amplitudes
    

In [11]:
def _generate_signals(num_signals, tfrac, prior_index, duration, draw_signal, sqrtSX):
    tmin = 1238166018
    dt = 10558244
    
    # These parameters describe background noise and data format
    writer_kwargs = {
                    "tstart": int(tmin + dt*tfrac), # 1238177971, # 1238166018,
                    "duration": 1800 * duration,
                    "detectors": "H1,L1",        
                    "sqrtSX": sqrtSX, # lambda: 10**stats.uniform(-26, -23).rvs(),          
                    "Tsft": 1800,             
                    "SFTWindowType": "tukey", 
                    "SFTWindowBeta": 0.01,
                    }

    # This class allows us to sample signal parameters from a specific population.
    # Implicitly, sky positions are drawn uniformly across the celestial sphere.
    # PyFstat also implements a convenient set of priors to sample a population
    # of isotropically oriented neutron stars.
    signal_parameters_generator = pyfstat.AllSkyInjectionParametersGenerator(
        priors={
            "tref": writer_kwargs["tstart"],
            "F0": {"uniform": {"low": 50.0, "high": 500.}},
            "Band": 359/1800 ,
            "F1": lambda: 10**stats.uniform(-12, 4).rvs(),
            "F2": 0,
            "h0": lambda: writer_kwargs["sqrtSX"] / stats.uniform(1, 10).rvs() if draw_signal else 0,
            **pyfstat.injection_parameters.isotropic_amplitude_priors,
        },
    )

    snrs = np.zeros(num_signals)

    for ind in range(num_signals):

        # Draw signal parameters.
        # Noise can be drawn by setting `params["h0"] = 0
        params = signal_parameters_generator.draw()
        
        DIRECTORY_TMP = f"~/Documents/TMP_NOT-SO-PURE_signals_true_{ind}_{prior_index}" if draw_signal else f"/TMP_5-generate-signals/signals_false_{ind}_{prior_index}"
        
        writer_kwargs["outdir"] = DIRECTORY_TMP
        
        writer = pyfstat.Writer(**writer_kwargs, **params)
        writer.make_data()
        
        paths = writer.sftfilepath.split(";")
        
        # Data can be read as a numpy array using PyFstat
        frequency, timestamps, amplitudes_0 = pyfstat.utils.get_sft_as_arrays(paths[0])
        frequency, timestamps, amplitudes_1 = pyfstat.utils.get_sft_as_arrays(paths[1])
        x = combine_amplitudes(amplitudes_0["H1"], amplitudes_1["L1"])
        
        return x

def generate_signals(num_signals=10, tfrac=0.9, prior_index=0,duration=4586, draw_signal=True, sqrtSX=1.E-23):

                     # Generate signals with parameters drawn from a specific population

    try: # 1238166018, 1248724262)
        _generate_signals(num_signals, tfrac, prior_index, duration, draw_signal, sqrtSX)
            
                
    except:
        print("LOL it failed")
        # Return raise error
        

In [13]:
x = _generate_signals(num_signals=1, tfrac=0.1, prior_index=0,duration=500, draw_signal=True, sqrtSX=1.E-23)

22-11-05 09:44:59.471 pyfstat.core INFO    : Creating Writer object...
22-11-05 09:44:59.472 pyfstat.utils.ephemeris INFO    : No /home/viktor/.pyfstat.conf file found. Will fall back to lal's automatic path resolution for files [earth00-40-DE405.dat.gz,sun00-40-DE405.dat.gz]. Alternatively, set 'earth_ephem' and 'sun_ephem' class options.
22-11-05 09:44:59.477 pyfstat.make_sfts INFO    : Generating SFTs with fmin=83.27062122306017, Band=0.19944444444444445
22-11-05 09:44:59.478 pyfstat.make_sfts INFO    : Checking if we can re-use injection config file...
22-11-05 09:44:59.478 pyfstat.make_sfts INFO    : ...OK: config file ~/Documents/TMP_NOT-SO-PURE_signals_true_0_0/PyFstat.cff already exists.
22-11-05 09:44:59.479 pyfstat.make_sfts INFO    : ...file contents unmatched, updating ~/Documents/TMP_NOT-SO-PURE_signals_true_0_0/PyFstat.cff.
22-11-05 09:44:59.479 pyfstat.make_sfts INFO    : Writing config file: ~/Documents/TMP_NOT-SO-PURE_signals_true_0_0/PyFstat.cff
22-11-05 09:44:59.479 

In [14]:
x.shape

(4, 360, 500)

In [4]:
num_signals = 50
num_priors = 500

sqrtX = np.logspace(-23.5, -22.5, num=num_priors)
tfrac = np.linspace(0, 1, num=num_priors)


In [ ]:
from joblib import Parallel, delayed

# generate_dataset(train_files, indx, train_test_label="train")
Parallel(n_jobs=6)(delayed(generate_signals)(num_signals=num_signals,tfrac=tf, prior_index=prior_index, duration=360, draw_signal=True, sqrtSX=sqrtX[prior_index])  for prior_index, tf in zip(range(num_priors), tfrac))
    

22-10-28 13:53:59.079 pyfstat INFO    : Running PyFstat version 1.18.1+1.73ad1acd.clean
22-10-28 13:53:59.084 pyfstat INFO    : Running PyFstat version 1.18.1+1.73ad1acd.clean
22-10-28 13:53:59.093 pyfstat INFO    : Running PyFstat version 1.18.1+1.73ad1acd.clean
22-10-28 13:53:59.107 pyfstat INFO    : Running PyFstat version 1.18.1+1.73ad1acd.clean
22-10-28 13:53:59.114 pyfstat INFO    : Running PyFstat version 1.18.1+1.73ad1acd.clean
22-10-28 13:53:59.125 pyfstat INFO    : Running PyFstat version 1.18.1+1.73ad1acd.clean
22-10-28 13:53:59.208 pyfstat.utils.importing INFO    : No $DISPLAY environment variable found, so importing matplotlib.pyplot with non-interactive 'Agg' backend.
22-10-28 13:53:59.209 pyfstat.utils.importing INFO    : No $DISPLAY environment variable found, so importing matplotlib.pyplot with non-interactive 'Agg' backend.
22-10-28 13:53:59.222 pyfstat.utils.importing INFO    : No $DISPLAY environment variable found, so importing matplotlib.pyplot with non-interactiv

# TEST

In [ ]:
paths =  ['./tmp/Signal_0_0/H-100_H1_1800SFT_Signal_0-1238166018-180000.sft', './tmp/Signal_0_0/L-100_L1_1800SFT_Signal_0-1238166018-180000.sft'] 
path = paths[0]
frequency, timestamps, amplitudes_0 = pyfstat.utils.get_sft_as_arrays(path)
path = paths[1]
frequency, timestamps, amplitudes_1 = pyfstat.utils.get_sft_as_arrays(path)
x = combine_amplitudes(amplitudes_0["H1"], amplitudes_1["L1"])
x.shape